In [9]:
from SPARQLWrapper import SPARQLWrapper, SPARQLWrapper2, JSON, TURTLE, XML, RDFXML
import pandas as pd
import matplotlib.pyplot as plt

In [10]:
import pprint
import csv
import ast

from collections import Counter
from operator import itemgetter

import sqlite3 as sql
import time

from importlib import reload
from shutil import copyfile

## Wikipedia: Les empereurs romains, page "Roman emperor"

*  https://en.wikipedia.org/wiki/Roman_emperor
*  https://en.wikipedia.org/wiki/List_of_Roman_emperors

## DBPedia : Roman emperor

* https://dbpedia.org/page/Roman_emperor (an instance of the class [Political System] (https://dbpedia.org/ontology/PoliticalSystem) )

* https://dbpedia.org/page/List_of_Roman_emperors 

### DBPedia SPARQL Endpoint

*  https://dbpedia.org/sparql
*  https://live.dbpedia.org/sparql (mis à jour en permanence, à utiliser pour les requêtes fédérées, i.e. la clause SERVICE)

Documentation : [SPARQL 1.1 Query Language](http://www.w3.org/TR/sparql11-query/)

### Auguste dans DBPedia

*  https://en.wikipedia.org/wiki/Augustus
*  https://dbpedia.org/page/Augustus

### Extrait de l'infobox de la page Wikipedia d'Auguste

*  https://en.wikipedia.org/wiki/Augustus

{{Infobox royalty
| name         = Augustus
| title        = [[Princeps]] Civitatis
| image        = Statue-Augustus.jpg
| image_size   = 250px
| alt          = Statue of Augustus
| caption      = [[Augustus of Prima Porta]], 1st century
| succession   = [[Roman emperor]]
| reign        = 16 January 27 BC – {{no wrap|19 August AD 14}}<br />(40 years and 7 months)
| successor    = [[Tiberius]]
| birth_name   = Gaius [[Octavii|Octavius]]
| birth_date   = 23 September 63 BC
| birth_place  = [[Rome]], [[Roman Italy|Italy]], [[Roman Republic]]
| death_date   = 19 August AD 14 (aged 75)
| death_place  = [[Nola]], Italy, [[Roman Empire]]
| burial_place = [[Mausoleum of Augustus]], [[Rome]]
| spouse       = {{plainlist|
* [[Claudia (wife of Octavian)|Claudia]] (42–40 BC; divorce)
* [[Scribonia (wife of Octavian)|Scribonia]] (40–38 BC; divorce)
* [[Livia]] (37 BC–AD 14; his death)}}
| issue        = {{ubl|[[Julia the Elder]]|[[Gaius Caesar]] (adopted)|[[Lucius Caesar]] (adopted)|[[Agrippa Postumus]] (adopted)|[[Tiberius]] (adopted)}}
| regnal name  = Imperator Caesar [[divi filius]] Augustus
| dynasty      = [[Julio-Claudian dynasty|Julio-Claudian]]
| father       = {{ubl|[[Gaius Octavius (father of Augustus)|Gaius Octavius]]|[[Julius Caesar]] (adoptive)}}
| mother       = [[Atia Balba Caesonia]]
| signature    = 
}}


## Premières requêtes SPARQL

In [18]:
###  Définir la variable contenant la requête SPARQL
# Requête : toutes les propriétés concernant Auguste

q_1 = """

PREFIX dbpedia: <http://dbpedia.org/resource/>
SELECT ?p ?o WHERE {
dbpedia:Augustus ?p ?o
}

"""

In [19]:
###  Requête: toutes les propriétés ayant un texte en anglais en objet


q_2 = """

PREFIX dbpedia: <http://dbpedia.org/resource/>
SELECT ?p ?o WHERE {
dbpedia:Augustus ?p ?o.
FILTER ( lang(?o) = "en" ) 
}

"""

In [5]:
###  Requête: compter et regrouper les propriétés

q_3 = """

PREFIX dbpedia: <http://dbpedia.org/resource/>
SELECT ?p (COUNT(*) as ?eff) WHERE {
dbpedia:Augustus ?p ?o.
}
GROUP BY ?p
ORDER BY DESC(?eff)

"""

In [6]:
### Requête: inspecter le contenu des différentes propriétés, en les analysant une par une

# Astuce: ouvrir une nouvelle fenêtre du point d'accès SPARQL DBPedia
# A partir du résultat de la requête précédente, copier coller les proriétés et exécuter

q_4 = """

PREFIX dbpedia: <http://dbpedia.org/resource/>
SELECT ?o WHERE {
dbpedia:Augustus <http://dbpedia.org/property/birthPlace> ?o.
}

"""

### Stocker les requêtes dans une base de données SQLite


*  Afin de ne pas surcharger le carnet, les requêtes SPARQL et les résultats sont stockées dans une base de données SQLite.
*  Elle se trouve dans le dossier /Users/sorayasanchez/Documents/jupyter_notebooks/HN_Cours/Exercices/sparql_queries.db.
*  On peut l'ouvrir et l'éditer avec le logiciel SQLiteStudio, cf. les indications de [cette page](http://phn-wiki.ish-lyon.cnrs.fr/doku.php?id=intro_histoire_numerique:modelisation_bases_donnees)

#### Manière de procéder

* Préparer la requête directement dans l'éditeur du point d'accès SPARQ original, dans ce cas: https://dbpedia.org/sparql
* Si nécessaire, valider et formater les requêtes dans ces plateformes:
  *  https://linked.bodc.ac.uk/validate/query
  *  http://sparql.org/query-validator.html
* Copier ensuite la requête dans une nouvelle ligne de la base de données, table _query_
* Ajouter un titre, un descriptif, l'adresse du point d'accès SPARQL dans la nouvelle ligne de la base de données
* Choisir la ligne à exécuter et retenir les numéro de sa clé primaire (_pk_query_)


In [20]:
### Inspecter les requêtes existantes

# connnexion à la base de données
original_db = '/Users/sorayasanchez/Documents/jupyter_notebooks/HN_Cours/Exercices/sparql_queries.db'

conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT pk_query, label, description, sparql_endpoint FROM query')
r_all = c.fetchall()

# fermer la connexion
conn.close()
r_all


OperationalError: no such table: query

In [15]:
### Définir la ligne de la base de données à utiliser (inspécter préalablement la base de données)
pk_query = ('4')

# connnexion à la base de données
original_db = '/Users/sorayasanchez/Documents/jupyter_notebooks/HN_Cours/Exercices/sparql_queries.db'

conn = sql.connect(original_db)

### exécuter la requëte sur la base de donées SQLite pour récupérer les valeurs que contient la ligne
c = conn.cursor()
c.execute('SELECT * FROM query WHERE pk_query = ?', pk_query)
rc = c.fetchone()

# fermer la connexion
conn.close()

# imprimer et inspecter le résultat
pprint.pprint(rc)

(4,
 'Inspecter propriété Birth Place',
 'Inspecter la propriété Birth Place',
 None,
 'https://dbpedia.org/sparql?default-graph-uri=http%3A%2F%2Fdbpedia.org&query=PREFIX++dbpedia%3A+%3Chttp%3A%2F%2Fdbpedia.org%2Fresource%2F%3E%0D%0A%0D%0ASELECT++%3Fo%0D%0AWHERE%0D%0A++%7B+dbpedia%3AAugustus%0D%0A++++++++++++++%3Chttp%3A%2F%2Fdbpedia.org%2Fproperty%2FbirthPlace%3E++%3Fo%0D%0A++%7D&format=text%2Fhtml&timeout=30000&signal_void=on&signal_unconnected=on',
 'PREFIX  dbpedia: <http://dbpedia.org/resource/>\n'
 '\n'
 'SELECT  ?o\n'
 'WHERE\n'
 '  { dbpedia:Augustus\n'
 '              <http://dbpedia.org/property/birthPlace>  ?o\n'
 '  }',
 None,
 None)


In [17]:
print(rc[4],'\n---\n',rc[5])

https://dbpedia.org/sparql?default-graph-uri=http%3A%2F%2Fdbpedia.org&query=PREFIX++dbpedia%3A+%3Chttp%3A%2F%2Fdbpedia.org%2Fresource%2F%3E%0D%0A%0D%0ASELECT++%3Fo%0D%0AWHERE%0D%0A++%7B+dbpedia%3AAugustus%0D%0A++++++++++++++%3Chttp%3A%2F%2Fdbpedia.org%2Fproperty%2FbirthPlace%3E++%3Fo%0D%0A++%7D&format=text%2Fhtml&timeout=30000&signal_void=on&signal_unconnected=on 
---
 PREFIX  dbpedia: <http://dbpedia.org/resource/>

SELECT  ?o
WHERE
  { dbpedia:Augustus
              <http://dbpedia.org/property/birthPlace>  ?o
  }
